# XGB Classifier with Filter-Based FS

Steven Sison | March 9, 2024

## Description

This document will be used to train a model using the reduced feature set obtain by using the wrapper-based method, forward feature selection. The model will be evaluated in terms of the usual metrics (accuracy, precision, F1-score, recall) as well as the training time. The model will also be stored for future evaluation purposes.

## Training the Model

### Preliminaries

#### 1. Loading the Dataset

In [2]:
import pandas as pd                     # For data transformation
import numpy as numpy                   # For scientific calculations
import seaborn as sns                   # For data visualizations
import matplotlib.pyplot as plt         # For plotting
import plotly.graph_objects as go       # For plotting
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, ConfusionMatrixDisplay
from xgboost import XGBClassifier, DMatrix, train
from sklearn.pipeline import Pipeline
import time
from datetime import datetime
import joblib
import os
import optuna
from sklearn.metrics import mean_squared_error # or any other metric
from sklearn.model_selection import train_test_split

dataset = pd.read_csv("../../../02_feature-engineering/final-datasets/binary_unbalanced_with_lexicalcontent.csv")      # Loading the dataset

dataset.head()

,url_type,blank_lines_count,blank_spaces_count,word_count,average_word_len,webpage_size,webpage_entropy,js_count,sus_js_count,js_eval_count,...,has_swf_in_string,has_cgi_in_string,has_php_in_string,has_abuse_in_string,has_admin_in_string,has_bin_in_string,has_personal_in_string,has_update_in_string,has_verification_in_string,url_scheme
0,1,14,36,0,0.000000,1186,5.269303,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,415,3952,11450,4.683930,26155,4.565537,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,5715,88450,420,5.833333,339327,4.656704,13,10,1,...,0,0,0,0,0,0,0,0,0,0
3,0,1461,6292,3049,4.515907,89249,5.149580,11,2,0,...,0,0,0,0,0,0,0,0,0,0
4,0,43,273,0,0.000000,1530,4.754726,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
dataset.head()

,url_type,blank_lines_count,blank_spaces_count,word_count,average_word_len,webpage_size,webpage_entropy,js_count,sus_js_count,js_eval_count,...,has_swf_in_string,has_cgi_in_string,has_php_in_string,has_abuse_in_string,has_admin_in_string,has_bin_in_string,has_personal_in_string,has_update_in_string,has_verification_in_string,url_scheme
0,1,14,36,0,0.000000,1186,5.269303,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,415,3952,11450,4.683930,26155,4.565537,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,5715,88450,420,5.833333,339327,4.656704,13,10,1,...,0,0,0,0,0,0,0,0,0,0
3,0,1461,6292,3049,4.515907,89249,5.149580,11,2,0,...,0,0,0,0,0,0,0,0,0,0
4,0,43,273,0,0.000000,1530,4.754726,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
x_train, x_test, y_train, y_test = train_test_split(dataset.drop(columns=['url_type']), dataset['url_type'], test_size = 0.2, random_state=42)

#### 2. Preprocessing (Balancing)

In [5]:
dataset['url_type'].value_counts()

url_type
0    120080
1     62334
Name: count, dtype: int64

#### 3. Removing Unnecessary Features

In [6]:
important_features_lexical = ['url_host_length',
                              'url_is_https',  
                              'url_num_periods',
                              'url_path_length',
                              'js_count',
                              'has_log_in_html',
                              'meta_tag_count',
                              'js_search_count',
                              'url_num_forward_slash',
                              'sus_js_count',
                              'js_link_count',
                              'webpage_entropy',
                              'url_num_ampersand',
                              'url_num_subdomain',
                              'webpage_size',
                              'has_free_in_html',
                              'has_php_in_string',
                              'url_num_of_hyphens',
                              'url_query_length',
                              'get_tld',
                              'url_scheme']

X_test_lexical_content = x_test[important_features_lexical]
X_train_lexical_content = x_train[important_features_lexical]

print("Lexical+Content-Based XGB Model has a total of "+str(len(important_features_lexical))+" features.")

Lexical+Content-Based XGB Model has a total of 21 features.


#### Hyper-parameter Optimization

In [7]:
# Define the objective function for Optuna
def objective_lexical_content(trial):
    # Define the search space for hyperparameters
    param = {
        'objective': 'binary:hinge',
        'eval_metric': 'error',
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'n_estimators': 100000, # Fix the boosting round and use early stopping
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 10.0),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 10.0),
        'lambda': trial.suggest_float('lambda', 0.1, 10.0),
        'alpha': trial.suggest_float('alpha', 0.0, 10.0),
    }
    
    # Split the data into further training and validation sets (three sets are preferable)
    train_data, valid_data, train_target, valid_target = train_test_split(X_train_lexical_content, y_train, test_size=0.2, random_state=42)
    
    # Convert the data into DMatrix format
    dtrain = DMatrix(train_data, label=train_target)
    dvalid = DMatrix(valid_data, label=valid_target)
    
    # Define the pruning callback for early stopping
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation-error')
    
    # Train the model with early stopping
    model = train(param, dtrain, num_boost_round=100000, evals=[(dvalid, 'validation')], early_stopping_rounds=100, callbacks=[pruning_callback])
    
    # Make predictions on the test set
    dtest = DMatrix(valid_data)
    y_pred = model.predict(dtest)
    
    # Calculate the root mean squared error
    error = mean_squared_error(valid_target, y_pred, squared=False)
    
    return error

# Create an Optuna study and optimize the objective function
study_lexical_content = optuna.create_study(direction='minimize')
study_lexical_content.optimize(objective_lexical_content, n_trials=100) # Control the number of trials

# Print the best hyperparameters and the best RMSE
best_params_lexical_content = study_lexical_content.best_params
best_error_lexical_content = study_lexical_content.best_value
print("Best Hyperparameters: ", best_params_lexical_content)
print("Best Error: ", best_error_lexical_content)

[I 2024-04-09 10:35:22,231] A new study created in memory with name: no-name-4c24c94b-83c5-47e0-a777-f8075edf5024


[0]	validation-error:0.65995

c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [10:35:22] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)



[1]	validation-error:0.65995
[2]	validation-error:0.20413
[3]	validation-error:0.17566
[4]	validation-error:0.12571
[5]	validation-error:0.12327
[6]	validation-error:0.11841
[7]	validation-error:0.11776
[8]	validation-error:0.11656
[9]	validation-error:0.11464
[10]	validation-error:0.09425
[11]	validation-error:0.09258
[12]	validation-error:0.09138
[13]	validation-error:0.08857
[14]	validation-error:0.08826
[15]	validation-error:0.08737
[16]	validation-error:0.08610
[17]	validation-error:0.08521
[18]	validation-error:0.08391
[19]	validation-error:0.08494
[20]	validation-error:0.08319
[21]	validation-error:0.08339
[22]	validation-error:0.08202
[23]	validation-error:0.07997
[24]	validation-error:0.07901
[25]	validation-error:0.07918
[26]	validation-error:0.07843
[27]	validation-error:0.07839
[28]	validation-error:0.07805
[29]	validation-error:0.07716
[30]	validation-error:0.07661
[31]	validation-error:0.07623
[32]	validation-error:0.07634
[33]	validation-error:0.07592
[34]	validation-er

[I 2024-04-09 10:35:30,054] Trial 0 finished with value: 0.2024281288757224 and parameters: {'eta': 0.24134137570737962, 'max_depth': 3, 'subsample': 0.605447303272991, 'colsample_bytree': 0.837582623308372, 'gamma': 3.6869901056534458, 'min_child_weight': 0.9105365592472066, 'lambda': 8.25265806591607, 'alpha': 2.6994109573039227}. Best is trial 0 with value: 0.2024281288757224.
c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [10:35:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


...

[0]	validation-error:0.65995


[I 2024-04-09 10:36:36,568] Trial 98 pruned. Trial was pruned at iteration 1.
c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [10:36:36] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation-error:0.65995


[I 2024-04-09 10:36:36,767] Trial 99 pruned. Trial was pruned at iteration 1.


Best Hyperparameters:  {'eta': 0.10031864027110182, 'max_depth': 9, 'subsample': 0.9231738329456532, 'colsample_bytree': 0.7841748561498321, 'gamma': 0.710300338796076, 'min_child_weight': 6.358346542127798, 'lambda': 8.191237945400758, 'alpha': 5.832972334974991}
Best Error:  0.18675797658082016


#### Model Training

In [8]:
from sklearn.model_selection import KFold
from xgboost import DMatrix, train

# Initialize CV
cv = KFold(n_splits=10, shuffle=True, random_state=1)

best_params_lexical_content['objective'] = 'binary:hinge'
best_params_lexical_content['eval_metric'] = 'error'

# Convert the data into DMatrix format
lexical_content_train = DMatrix(X_train_lexical_content, label=y_train)
lexical_content_valid = DMatrix(X_test_lexical_content, label=y_test)

# Train the Model
xgb_classifier = train(best_params_lexical_content, lexical_content_train, num_boost_round=1000)
y_pred_lexical_content = xgb_classifier.predict(lexical_content_valid)

print("Model training done.")

Model training done.


In [9]:
# Classification Report
print(classification_report(y_test, y_pred_lexical_content))

              precision    recall  f1-score   support

           0       0.97      0.98      0.97     23942
           1       0.96      0.94      0.95     12541

    accuracy                           0.96     36483
   macro avg       0.96      0.96      0.96     36483
weighted avg       0.96      0.96      0.96     36483



#### Practical Evaluation

In [10]:
# Dumping the model
joblib.dump(xgb_classifier, 'xgb_filter_lexical-content.sav')

['xgb_filter_lexical-content.sav']

In [14]:
import content_generator_filter
import feature_generation_content_function_htmlin
import time

def xgb_predict_maliciousness(url):

    numerical_values = content_generator_filter.feature_generator(url)
    # print(numerical_values)
    numerical_values = DMatrix(numerical_values)

    match xgb_classifier.predict(numerical_values):
        case 0:
            return "Benign"
        case 1:
            return "Malware"
        case 2:
            return "Phishing"
        case 3:
            return "Defacement"

url = "www.facebook.com/"
print("Current URL: "+url)

print("------------- Filter-Based (Lexical + Content) -------------")
for i in range(15):
    start = time.perf_counter()
    prediction = xgb_predict_maliciousness(url)
    end = time.perf_counter()
    print("Trial "+str(i))
    print(prediction)
    print(end-start)

Current URL: www.facebook.com/
------------- Filter-Based (Lexical + Content) -------------
Trial 0
Benign
0.5154141000239179
Trial 1
Benign
0.4878694000071846
Trial 2
Benign
0.42477990000043064
Trial 3
Benign
0.4510425999760628
Trial 4
Benign
0.43153169995639473
Trial 5
Benign
0.4045920000062324
Trial 6
Benign
0.43066009995527565
Trial 7
Benign
0.4085478000342846
Trial 8
Benign
0.448262199992314
Trial 9
Benign
0.5521377000259236
Trial 10
Benign
0.4245381999644451
Trial 11
Benign
0.43366730003617704
Trial 12
Benign
0.5210599999991246
Trial 13
Benign
0.5085515999817289
Trial 14
Benign
0.4594624000019394


#### Evaluation

In [12]:
'''# Confusion Matrix for 12 Features
cm_up = confusion_matrix(y_test, y_pred, labels=xgb_classifier.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix = cm_up, display_labels = xgb_classifier.classes_)
disp.plot()
plt.show()'''

'# Confusion Matrix for 12 Features\ncm_up = confusion_matrix(y_test, y_pred, labels=xgb_classifier.classes_)\ndisp = ConfusionMatrixDisplay(confusion_matrix = cm_up, display_labels = xgb_classifier.classes_)\ndisp.plot()\nplt.show()'

In [13]:
'''# Cross Validation Score
scores = cross_val_score(XGBClassifier(**params_gbm),
                        X_train, y_train, scoring='accuracy', cv=cv).mean()

print(scores)'''

"# Cross Validation Score\nscores = cross_val_score(XGBClassifier(**params_gbm),\n                        X_train, y_train, scoring='accuracy', cv=cv).mean()\n\nprint(scores)"